In [ ]:
from fastai import *

from fastai.tabular import *

import os

c_d = os.getcwd();path

path

path = '/notebooks/tl/'

df = pd.read_csv(path+'Preprocessed_data/more_features_train.csv')

import gc
del df
gc.collect()

cont_names = ['char_count','inc_veh_age','inc_totallosspoints','unique_word_count','word_count','inc_veh_mileage_aaa',
              'inc_odomread','inc_salvage_aaa','inc_lossestimate','inc_towpickuplocation_aaaid']    
cat_names = [col for col in train.columns if col not in cont_names]
procs = [FillMissing, Categorify, Normalize]

dep_var = 'inc_totalloss'

sel_features = ['claim_losscause',
 'word_density',
 'char_count',
 'inc_veh_age',
 'Year_Day_loss',
 'Year_Day_report',
 'inc_veh_year',
 'Hour_loss',
 'inc_totallosspoints',
 'day_diff',
 'inc_vehicleoperable',
 'unique_word_count',
 'Hour_report',
 'word_count',
 'inc_veh_mileage_aaa',
 'claim_reportedbytype',
 'Month_Day_loss',
 'Month_Day_report',
 'exp_exposuretier',
 'claimant_cont_injured_aaa',
 'claim_addr_state',
 'inc_driverrelation',
 'Weekday_loss',
 'inc_allowdirecttow_aaa',
 'inc_veh_style',
 'exp_cov_deductible',
 'Weekday_report',
 'claim_faultrating',
 'exp_jurisdictionstate',
 'inc_vehicleparked',
 'inc_vehicleincidentseqnumber_aaa',
 'inc_vehicletype',
 'inc_odomread',
 'claim_coverageinquestion',
 'inc_veh_state',
 'inc_drnshopselectedind_aaa',
 'claim_fault',
 'inc_veh_ubi_aaa',
 'inc_salvage_aaa',
 'inc_atleast3bodypanaeldamage_aaa',
 'inc_childseat_aaa',
 'inc_airbagsdeployed',
 'inc_thftstolenind',
 'Year_loss',
 'inc_lossestimate',
 'Year_report',
 'inc_directtowselectedind_aaa',
 'claim_jurisdictionstate',
 'exp_coveragesubtype',
 'claim_claimtier',
 'exp_exposuresubrostatus_aaa',
 'exp_state',
 'inc_towpickuplocation_aaaid',
 'exp_validationlevel',
 'inc_vehicleage5years',
 'inc_drndocverbaseind_aaa',
 'inc_childseatinusenum_aaa',
 'inc_speed',
 'inc_damagetowindshieldglass_aaa',
 'inc_previnsvehrepaired_aaa',
 'inc_previnsvehrental_aaa',
 'inc_bodyshopselected',
 'inc_mileage100k',
 'inc_inspselected_aaa',
 'inc_vehfireinvolved_aaa',
 'exp_cov_zeroglassdeduct_aaa',
 'claim_liabilitycode_aaa']

train = df[sel_features].copy()
train['inc_totalloss']=df.inc_totalloss.copy()

cat_names.remove('inc_totalloss')

import gc
del df
gc.collect()

cat_list = []
for col in train.columns:
    if train[col].dtype=='O':
        print(col)
        cat_list.append(col)

print([col for col in train.columns if train[col].nunique() >100])

train.inc_towpickuplocation_aaaid.nunique()

data = (TabularList.from_df(train,cat_names=cat_names, cont_names=cont_names, procs=procs)
                        .random_split_by_pct(0.15)
                           .label_from_df(cols=dep_var)
                           .databunch())

data.show_batch(rows=10)

learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

learn.fit_one_cycle(1,1e-2)

data.train_ds[999999]

path

text01= pd.read_csv(path+'Preprocessed_data/text_clean.csv')

text01.shape

text01.head()

text02 = pd.read_csv(path+'Preprocessed_data/text_totaloss.csv')

text02.head()

del text01
gc.collect()

text01.head()

df = pd.read_csv(path+'Preprocessed_data/more_features_train.csv')

df.inc_veh_make_new[:10]

df.inc_veh_model[:10]

df.inc_veh_year.describe()

pd.set_option('display.float_format', lambda x: '%.3f' % x)


df.loc[df.inc_veh_year <0,'inc_veh_year']=2017

text02['vehicle_properties'] = 'Vehicle manufacturer is '+df['inc_veh_make_new'].astype(str)+'. Vehicle model is '+ df['inc_veh_model'].astype(str)+'. Vehicle year is '+ df['inc_veh_year'].astype(str) + '. Vehicle age is '+ df['inc_veh_age'].astype(str)

text02.vehicle_properties[:10].tolist()

text02.head()

text02['text'] = text02['text_combines'].astype(str)+ ' ' + text02['vehicle_properties'].astype(str)

text02['text'].tolist()

text02.drop(['Unnamed: 0','text_combines','inc_veh_properties'],axis=1,inplace=True)

text02.drop('vehicle_properties',axis=1,inplace=True)

text02.text.to_csv(path+'Preprocessed_data/text_only.csv',index=False)

from fastai.text import *

data_lm = TextDataBunch.from_csv(path,'Preprocessed_data/text_comb.csv')

data_lm.save()

data = TextDataBunch.load(path)

data.show_batch()

data.vocab.itos[:10]

data.train_ds[0][0]

data.train_ds[0][0].data[:10]

bs=32

data = (TextList.from_csv(path, 'Preprocessed_data/text_comb.csv', cols='text')
                .split_from_df(col=2)
                .label_from_df(cols=0)
                .databunch())

data_lm.save('tmp_lm')

data_lm = TextLMDataBunch.load(path, 'tmp_lm', bs=bs)

learn = language_model_learner(data_lm, pretrained_model=URLs.WT103, drop_mult=0.3)

learn.lr_find()

learn.recorder.plot(skip_end=15)

learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

learn.save('fit_head')

learn.load('fit_head');

learn.unfreeze()
learn.fit_one_cycle(10, 1e-3, moms=(0.8,0.7))

learn.save('fine_tuned')

learn.load('fine_tuned');

learn.predict('IV was parked in sideway ', 100, temperature=1.1, min_p=0.001)

learn.predict('Vehicle Manufactures  ', 100, temperature=1.1, min_p=0.001)

learn.save_encoder('fine_tuned_enc')

## Classifier

data_clas = (TextList.from_folder(path, vocab=data_lm.vocab)
             #grab all the text files in path
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             .label_from_folder(classes=['0', '1'])
             #remove docs with labels not in above list (i.e. 'unsup')
             .filter_missing_y()
             #label them all with their folders
             .databunch(bs=bs))
data_clas.save('tmp_clas')

data = (TextList.from_csv(path, 'Preprocessed_data/text_comb.csv', cols='text')
                .databunch(bs=bs))

data_clas = TextClasDataBunch.load(path, 'tmp_clas', bs=bs)
data_clas.show_batch()
